# Comparing LLM Path Extractors for Knowledge Graph Construction

In this notebook, we'll compare three different LLM Path Extractors from llama_index:
1. SimpleLLMPathExtractor
2. SchemaLLMPathExtractor
3. DynamicLLMPathExtractor (New)

We'll use a Wikipedia page as our test data and visualize the resulting knowledge graphs using Pyvis.

## Setup and Imports

In [ ]:
!pip install llama_index pyvis wikipedia

In [ ]:
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

import wikipedia

import os

In [ ]:
import nest_asyncio

nest_asyncio.apply()

## Set up LLM Backend

In [ ]:
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_KEY_HERE"

# Set up global configurations
llm = OpenAI(temperature=0.0, model="gpt-3.5-turbo")

Settings.llm = llm
Settings.chunk_size = 2048
Settings.chunk_overlap = 20

## Fetch Some Raw Text from Wikipedia

In [ ]:
def get_wikipedia_content(title):
    try:
        page = wikipedia.page(title)
        return page.content
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation page. Options: {e.options}")
    except wikipedia.exceptions.PageError:
        print(f"Page '{title}' does not exist.")
    return None

In [ ]:
wiki_title = "Barack Obama"
content = get_wikipedia_content(wiki_title)

if content:
    document = Document(text=content, metadata={"title": wiki_title})
    print(
        f"Fetched content for '{wiki_title}' (length: {len(content)} characters)"
    )
else:
    print("Failed to fetch Wikipedia content.")

Fetched content for 'Barack Obama' (length: 83713 characters)


## 1. SimpleLLMPathExtractor

In [ ]:
kg_extractor = SimpleLLMPathExtractor(
    llm=llm, max_paths_per_chunk=20, num_workers=4
)

simple_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

simple_index.property_graph_store.save_networkx_graph(
    name="./SimpleGraph.html"
)
simple_index.property_graph_store.get_triplets(
    entity_names=["Barack Obama", "Obama"]
)[:5]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting paths from text: 100%|██████████| 11/11 [00:08<00:00,  1.26it/s]


[(EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '592e1402-edf3-419a-8f65-04410eb67df4'}, name='Obama'),
  Relation(label='Received', source_id='Obama', target_id='Ambassador of humanity award', properties={'title': 'Barack Obama', 'triplet_source_id': '592e1402-edf3-419a-8f65-04410eb67df4'}),
  EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '592e1402-edf3-419a-8f65-04410eb67df4'}, name='Ambassador of humanity award')),
 (EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '592e1402-edf3-419a-8f65-04410eb67df4'}, name='Obama'),
  Relation(label='Ordered', source_id='Obama', target_id='Campaign of u.s. airstrikes against isil', properties={'title': 'Barack Obama', 'triplet_source_id': '3fca0252-c372-42bb-8e2f-39a8b388523f'}),
  EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '3fca

## 2. DynamicLLMPathExtractor

### Without intial ontology :
Here, we let the LLM define the ontology on the fly, giving it full freedom to label the nodes as it best sees fit.

In [ ]:
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    allowed_entity_types=None,  # Let the LLM infer entities and their labels (types) on the fly
    allowed_relation_types=None,  # Let the LLM infer relationships on the fly
)

dynamic_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

dynamic_index.property_graph_store.save_networkx_graph(
    name="./DynamicGraph.html"
)
dynamic_index.property_graph_store.get_triplets(
    entity_names=["Barack Obama", "Obama"]
)[:5]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting and inferring knowledge graph from text: 100%|██████████| 11/11 [00:31<00:00,  2.90s/it]


[(EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'e5ee0bd7-38cb-4367-9ded-9d1e60f5f94a'}, name='Obama'),
  Relation(label='CLAIMED', source_id='Obama', target_id='U.S.  cannot succeed in Afghanistan or secure our homeland unless we change our Pakistan policy ', properties={'title': 'Barack Obama', 'triplet_source_id': 'b1645295-dc15-46a8-adf2-5b37b7c08b4f'}),
  EntityNode(label='EVENT', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'b1645295-dc15-46a8-adf2-5b37b7c08b4f'}, name='U.S. "cannot succeed in Afghanistan or secure our homeland unless we change our Pakistan policy"')),
 (EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'e5ee0bd7-38cb-4367-9ded-9d1e60f5f94a'}, name='Obama'),
  Relation(label='HAS_DAUGHTER', source_id='Obama', target_id='Malia Ann', properties={'title': 'Barack Obama', 'triplet_source_id': '72c882f6-24c0-4069-8aca-6a769a49724f'}),


### With initial ontology for guided KG extraction : 
Here, we have partial knowledge of what we want to detect, we know the article is about Barack Obama, so we define some entities and relations that could help guide the LLM in the labeling process as it detects the entities and relations. This doesn't guarantee that the LLM will use them, it simply guides it and gives it some ideas. It will still be up to the LLM to decide whether it uses the entities and relations we provide or not. 

In [ ]:
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    allowed_entity_types=["POLITICIAN", "POLITICAL_PARTY"],
    allowed_relation_types=["PRESIDENT_OF", "MEMBER_OF"],
)

dynamic_index_2 = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

dynamic_index_2.property_graph_store.save_networkx_graph(
    name="./DynamicGraph_2.html"
)
dynamic_index_2.property_graph_store.get_triplets(
    entity_names=["Barack Obama", "Obama"]
)[:5]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting and inferring knowledge graph from text: 100%|██████████| 11/11 [00:33<00:00,  3.06s/it]


[(EntityNode(label='POLITICIAN', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '06aa98e3-94eb-4be8-bc1b-a18af1f03648'}, name='Obama'),
  Relation(label='CLAIMED', source_id='Obama', target_id='U.S.  cannot succeed in Afghanistan or secure our homeland unless we change our Pakistan policy ', properties={'title': 'Barack Obama', 'triplet_source_id': '96dd0a8f-e635-4183-bbfc-6445ce0ec9c5'}),
  EntityNode(label='STATEMENT', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '96dd0a8f-e635-4183-bbfc-6445ce0ec9c5'}, name='U.S. "cannot succeed in Afghanistan or secure our homeland unless we change our Pakistan policy"')),
 (EntityNode(label='POLITICIAN', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'c1ab8269-64fc-42ea-b8c4-2f2592120444'}, name='Barack Obama'),
  Relation(label='RECEIVED', source_id='Barack Obama', target_id='Ripple of Hope Award', properties={'title': 'Barack Obama', 'triplet_source_id': 'c1ab8269

# 3 - SchemaLLMPathExtractor

In [ ]:
kg_extractor = SchemaLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    strict=False,  # Set to False to showcase why it's not going to be the same as DynamicLLMPathExtractor
    possible_entities=None,  # USE DEFAULT ENTITIES (PERSON, ORGANIZATION... etc)
    possible_relations=None,  # USE DEFAULT RELATIONSHIPS
    num_workers=4,
)

schema_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

schema_index.property_graph_store.save_networkx_graph(
    name="./SchemaGraph.html"
)
schema_index.property_graph_store.get_triplets(
    entity_names=["Barack Obama", "Obama"]
)[:5]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting paths from text with schema: 100%|██████████| 11/11 [00:19<00:00,  1.76s/it]


[(EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'f753a293-1e03-4f00-9683-28424a6ed11d'}, name='Barack Obama'),
  Relation(label='WORKED_ON', source_id='Barack Obama', target_id='Hurricane Irma', properties={'title': 'Barack Obama', 'triplet_source_id': 'f753a293-1e03-4f00-9683-28424a6ed11d'}),
  EntityNode(label='EVENT', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'f753a293-1e03-4f00-9683-28424a6ed11d'}, name='Hurricane Irma')),
 (EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '8ba7cd25-cce3-4c93-b69f-144c8f40fd34'}, name='Obama'),
  Relation(label='WORKED_ON', source_id='Obama', target_id='Iran nuclear talks', properties={'title': 'Barack Obama', 'triplet_source_id': '8ba7cd25-cce3-4c93-b69f-144c8f40fd34'}),
  EntityNode(label='EVENT', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '8ba7cd25-cce3-4c93-b69f-144c8f40fd34'},

## Comparison and Analysis

Let's compare the results of the three extractors:

1. **SimpleLLMPathExtractor**: This extractor creates a basic knowledge graph without any predefined schema. It may produce a larger number of diverse relationships but might lack consistency in entity and relation naming.


3. **DynamicLLMPathExtractor**: 
    - This new extractor combines the flexibility of SimpleLLMPathExtractor with some initial guidance from a schema. It can expand beyond the initial entity and relation types, potentially producing a rich and diverse graph while maintaining some level of consistency. 
    - Not giving it any entities or relations to start with in the input gives the LLM complete freedom to infer the schema on the fly as it best sees fit. This is going to vary based on the LLM and the temperature used.

3. **SchemaLLMPathExtractor**: With a predefined schema, this extractor produces a more structured graph. The entities and relations are limited to those specified in the schema, which can lead to a more consistent but potentially less comprehensive graph. Even if we set "strict" to false, the extracted KG Graph doesn't reflect the LLM's pursuit of trying to find new entities and types that fall outside of the input schema's scope.


## Key observations:

- The SimpleLLMPathExtractor graph might have the most diverse set of entities and relations.
- The SchemaLLMPathExtractor graph should be the most consistent but might miss a lot of relationships that don't fit the predefined schema, even if we don't impose a strict validation of the schema.
- The DynamicLLMPathExtractor graph should show a balance between diversity and consistency, potentially capturing important relationships that the schema-based approach might miss while still maintaining some structure.

## The choice between these extractors depends on the specific use case:

- Use SimpleLLMPathExtractor for exploratory analysis where you want to capture a wide range of potential relationships for RAG applications, without caring about the entity types.
- Use SchemaLLMPathExtractor when you have a well-defined domain and want to ensure consistency in the extracted knowledge.
- Use DynamicLLMPathExtractor when you want a balance between structure and flexibility, allowing the model to discover new entity and relation types while still providing some initial guidance. This one is especially useful if you want a KG with labeled (typed) entities but don't have an input Schema (or you've partially defined the schema as a starting base).